In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics.pairwise import sigmoid_kernel, rbf_kernel, polynomial_kernel
from sklearn import neighbors
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


## 1. Load Data

In [2]:
# Load the features
X1 = np.loadtxt(open("AD_MRI_DATA/1-CN_AD/1.1-MRI_CorticalThickness.csv","rb"),delimiter=",",skiprows=0)
X1 = np.array(X1, dtype=float)
print("X1:" + str(X1.shape))

X2 = np.loadtxt(open("AD_MRI_DATA/1-CN_AD/1.2-MRI_SurfaceArea.csv","rb"),delimiter=",",skiprows=0)
X2 = np.array(X2, dtype=float)
print("X2:" + str(X2.shape))

X3 = np.loadtxt(open("AD_MRI_DATA/1-CN_AD/1.3-MRI_Volume.csv","rb"),delimiter=",",skiprows=0)
X3 = np.array(X3, dtype=float)
print("X3:" + str(X3.shape))

# Load the labels
with open('AD_MRI_DATA/1-CN_AD/1-Phenotype.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    c1 = [row[1]for row in reader]
    
del c1[0]
y = np.array(c1, dtype=int)
print("y:" + str(y.shape))


X1:(134, 136)
X2:(134, 68)
X3:(134, 109)
y:(134,)


In [3]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
X1 = min_max_scaler.fit_transform(X1)
X2 = min_max_scaler.fit_transform(X2)
X3 = min_max_scaler.fit_transform(X3)

## 2. Lasso + MKSVM

### 2.1 lasso

In [4]:

w11, w12, w13 = 0, 0, 1
w21, w22, w23 = 0, 0, 1
w31, w32, w33 = 0, 0, 1

In [5]:
# Feature 1: CorticalThickness
model = linear_model.LassoCV()
#alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X1, y)

print("alpha: " + str(model.alpha_))
#print(lasso.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected1 = selector.transform(X1)
print("X_selected1: " + str(X_selected1.shape))


alpha: 0.002004268249900883
X_selected1: (134, 59)


In [6]:
# Feature 2：SurfaceArea
model = linear_model.LassoCV()
#alphas=[1,0.1,0.01,0.005,0.001,0.0001])
model.fit(X2, y)

print("alpha: " + str(model.alpha_))
#print(lasso.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected2 = selector.transform(X2)
print("X_selected2: " + str(X_selected2.shape))


alpha: 0.0024672277127070746
X_selected2: (134, 35)


In [7]:
# Feature 3：Volume
model = linear_model.LassoCV()
#alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X3, y)

print("alpha: " + str(model.alpha_))
#print(lasso.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected3 = selector.transform(X3)
print("X_selected3: " + str(X_selected3.shape))


/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.861e-03, tolerance: 2.421e-03
  model = cd_fast.enet_coordinate_descent(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.022e-02, tolerance: 2.421e-03
  model = cd_fast.enet_coordinate_descent(
/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

alpha: 0.0044870812378087115
X_selected3: (134, 25)


### 2.2 Multi-core SVM

In [8]:
# Calculate the kernel matrix of CorticalThickness
sig_mat1 = sigmoid_kernel(X_selected1)
rbf_mat1 = rbf_kernel(X_selected1)
poly_mat1 = polynomial_kernel(X_selected1)


# Calculate the kernel matrix of SurfaceArea
sig_mat2 = sigmoid_kernel(X_selected2)
rbf_mat2 = rbf_kernel(X_selected2)
poly_mat2 = polynomial_kernel(X_selected2)


# Calculate the kernel matrix of Volume
sig_mat3 = sigmoid_kernel(X_selected3)
rbf_mat3 = rbf_kernel(X_selected3)
poly_mat3 = polynomial_kernel(X_selected3)


kernel_mat1 = w11 * sig_mat1 + w12 * rbf_mat1 + w13 * poly_mat1
kernel_mat2 = w21 * sig_mat2 + w22 * rbf_mat2 + w23 * poly_mat2
kernel_mat3 = w31 * sig_mat3 + w32 * rbf_mat3 + w33 * poly_mat3


best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        kernel_mat = w1 * kernel_mat1 + w2 * kernel_mat2 + w3 * kernel_mat3
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, kernel_mat, y, cv=5, scoring="accuracy")
        acc_avg = np.sum(acc)/5
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.1 w2:0.30000000000000004 w3:0.6
score:0.9923076923076923


In [9]:
kernel_mat = best_w1 * kernel_mat1 + best_w2 * kernel_mat2 + best_w3 * kernel_mat3
y_pred = cross_val_predict(clf, kernel_mat, y, cv=5)
auc = cross_val_score(clf, kernel_mat, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[45  1]
 [ 0 88]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        88
           1       1.00      0.98      0.99        46

    accuracy                           0.99       134
   macro avg       0.99      0.99      0.99       134
weighted avg       0.99      0.99      0.99       134

auc: 1.0



## 3. ElasticNet+MKSVM

### 3.1 ElasticNet

In [10]:
w11, w12, w13 = 0, 0, 1
w21, w22, w23 = 0, 0, 1
w31, w32, w33 = 0, 0, 1


In [11]:
# Feature 1：CorticalThickness
model = ElasticNetCV(random_state=0)
#,alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X1, y)
print("alpha: " + str(model.alpha_))
#print(els.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected1 = selector.transform(X1)
print("X_selected1: " + str(X_selected1.shape))


alpha: 0.0032514475413349913
X_selected1: (134, 37)


In [12]:
# Feature 2:SurfaceArea
model = ElasticNetCV(random_state=0)
#,alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X2, y)
print("alpha: " + str(model.alpha_))
#print(els.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected2 = selector.transform(X2)
print("X_selected2: " + str(X_selected2.shape))


alpha: 0.004601889644787304
X_selected2: (134, 23)


In [13]:
# Feature 3：Volume
model = ElasticNetCV(random_state=0)
#,alphas=[1,0.1,0.01,0.005,0.001,0.0001]
model.fit(X3, y)
print("alpha: " + str(model.alpha_))
#print(els.coef_)

selector = SelectFromModel(estimator = model, prefit = True)
selector.get_support()
X_selected3 = selector.transform(X3)
print("X_selected3: " + str(X_selected3.shape))


/Users/zhengwentao/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.850e-03, tolerance: 2.421e-03
  model = cd_fast.enet_coordinate_descent(


alpha: 0.007805268862498193
X_selected3: (134, 24)


### 3.2 Multi-core SVM

In [14]:
# Calculate the kernel matrix of CorticalThickness
sig_mat1 = sigmoid_kernel(X_selected1)
rbf_mat1 = rbf_kernel(X_selected1)
poly_mat1 = polynomial_kernel(X_selected1)


# Calculate the kernel matrix of SurfaceArea
sig_mat2 = sigmoid_kernel(X_selected2)
rbf_mat2 = rbf_kernel(X_selected2)
poly_mat2 = polynomial_kernel(X_selected2)


# Calculate the kernel matrix of Volume
sig_mat3 = sigmoid_kernel(X_selected3)
rbf_mat3 = rbf_kernel(X_selected3)
poly_mat3 = polynomial_kernel(X_selected3)


kernel_mat1 = w11 * sig_mat1 + w12 * rbf_mat1 + w13 * poly_mat1
kernel_mat2 = w21 * sig_mat2 + w22 * rbf_mat2 + w23 * poly_mat2
kernel_mat3 = w31 * sig_mat3 + w32 * rbf_mat3 + w33 * poly_mat3


best_score = 0
best_w1 = 0
best_w2 = 0
best_w3 = 0

for i in range(11):
    for j in range(11):
        w1 = i * 0.1
        w2 = j * 0.1
        w3 = 1 -w1-w2
        if w1<-0.01 or w2<-0.01 or w3<-0.01:
            break
        kernel_mat = w1 * kernel_mat1 + w2 * kernel_mat2 + w3 * kernel_mat3
        clf = SVC(kernel='precomputed', probability=True)
        acc = cross_val_score(clf, kernel_mat, y, cv=5, scoring="accuracy")
        acc_avg = (acc[0] + acc[1] + acc[2])/3
        if acc_avg > best_score:
            best_w1 = w1
            best_w2 = w2
            best_w3 = w3
            best_score = acc_avg
        
print("w1:"+str(best_w1), "w2:"+str(best_w2), "w3:"+str(best_w3))
print("score:"+str(best_score))

w1:0.1 w2:0.30000000000000004 w3:0.6
score:1.0


In [15]:
kernel_mat = best_w1 * kernel_mat1 + best_w2 * kernel_mat2 + best_w3 * kernel_mat3
y_pred = cross_val_predict(clf, kernel_mat, y, cv=5)
auc = cross_val_score(clf, kernel_mat, y, cv=5, scoring="roc_auc")
print(confusion_matrix(y, y_pred,labels=[1,0]))
print(classification_report(y, y_pred))
auc_avg = (auc[0]+auc[1]+auc[2]+auc[3]+auc[4])/5
print("auc: "+str(auc_avg)+"\n")

[[45  1]
 [ 0 88]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        88
           1       1.00      0.98      0.99        46

    accuracy                           0.99       134
   macro avg       0.99      0.99      0.99       134
weighted avg       0.99      0.99      0.99       134

auc: 1.0

